# 実験環境を作成する

実験環境リポジトリの新規作成と実験記録に必要な実験パッケージの用意によって実験環境の用意をサポートします。  
実験環境リポジトリは、研究プロジェクトのリポジトリとは別のリポジトリですが、研究リポジトリのexperimental-environmentsフォルダ配下に子関係として追加されます。  
上から順番に実行してください。  
2回目以降の実行の場合、このセルが選択された状態で画面上部に表示される以下のボタンをクリックしてから実行して下さい。  
![UnfreezeBotton](https://raw.githubusercontent.com/NII-DG/workflow-template/develop/sections/images/unfreeze_button.png)

## 1. 作成したい実験環境リポジトリ名を入力する

以下のセルを実行し、表示されるテキストボックスに作成したい実験環境リポジトリ名を半角英数字で入力してください。  
データガバナンス機能にすでに存在するリポジトリ名のリポジトリは作成いただけません。分かりやすい名前をご記入ください。  
入力した実験環境リポジトリ名の履歴をこのノートブックに保存したい場合は、実行後にセルの左側に表示されるピンマークをクリックしてください。  
※入力値に誤りがある場合、もう一度実行することで訂正ができます。

In [ ]:
import re
from IPython.display import clear_output

# GINサーバのものに合わせたバリデーションルールを設定
validation = re.compile(r'[a-z|A-Z|0-9|\-|_|.]+')

print('作成したい実験リポジトリ名を半角英数字で入力してください。')
while True:
    repository_title = input("リポジトリ名：")
    if validation.fullmatch(repository_title):
        break
    else:
        clear_output()
        print('リポジトリ名は英数字、および"-", "_", "."のみで入力してください。')

clear_output()
print("作成したリポジトリ名：", repository_title)
print('この実験名で処理を進めます。変更したい場合は、このセルをもう一度実行することで訂正ができます。')


## 2. 実験環境リポジトリを作成する

先ほど入力いただいた名前の実験環境リポジトリをデータガバナンス機能に作成します。  
そのために、以下のセルを実行し、表示されるテキストボックスにデータガバナンス機能のユーザ名とパスワードを入力してください。  
この情報は認証のために利用します。

In [ ]:
import json
import os

path_flow_root = '/home/jovyan/WORKFLOW/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
# リポジトリ作成のためのユーザ認証
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    tokens = response.json()

    # 既存のトークンがなければ作成する
    if len(tokens) < 1:
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))

    if response.status_code == HTTPStatus.OK or HTTPStatus.CREATED:
        tokens = response.json()
        clear_output()
        print("認証が正常に完了しました。次の手順へお進みください。")
        break
    else:
        clear_output()
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")

In [ ]:
# GIN APIでリポジトリを作成する
import requests

create_repo = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/repos?token=' + tokens[0]['sha1'],
                data={
                    "name": repository_title,
                    "auto_init": False
                }
                )
print(create_repo)

## 3. 実験記録管理のための準備を行う

### - 3.1 実験環境リポジトリに実験パッケージを用意する

In [ ]:
tmp_path = '/home/jovyan/.tmp/' + repository_title + '/'

In [ ]:
%%bash
#!/bin/bash
if [ ! -d ~/.tmp/ ]; then
    # .tmpフォルダが無ければ作成する
    mkdir ~/.tmp
fi

if [ ! -d $tmp_path ]; then
    # 実験タイトル名のフォルダが無ければ作成する
    mkdir $tmp_path
fi

In [ ]:
import os

os.chdir(path_flow_root)
from util.scripts import utils

# DS構成のスキーム名をパラメタファイルから取得する
monitoring_params = {}
with open(utils.fetch_monitoring_param_file_path(), mode='r') as f:
    monitoring_params = json.load(f)
    
scheme_name = monitoring_params['datasetStructure']

# urlを生成する
remote_url = params['siblings']['ginSsh']+name+'/'+repository_title+'.git'
remote_http_url = params['siblings']['ginHttp']+name+'/'+repository_title+'.git'

# 実験パッケージを用意する
!cp -r ~/WORKFLOW/PACKAGE/base/. $tmp_path
!cp -r ~/WORKFLOW/PACKAGE/scheme/$scheme_name/. $tmp_path

# dataset化してsiblingを設定する
%cd $tmp_path
!datalad create --force .
!datalad siblings add -s "gin" --url $remote_url

### - 3.2 必要に応じてDockerfileを編集する

実験環境の構築のためにDockerfileの編集が必要な場合は、以下のセルを実行すると表示されるリンクからDockerfileを編集いただけます。

In [ ]:
import os
url = params['rcosBinderUrl'] + os.environ["JUPYTERHUB_SERVICE_PREFIX"] + 'edit/.tmp/' + repository_title + '/Dockerfile'
print(url)

### - 3.3 実験パッケージをデータガバナンス機能に同期する

In [ ]:
# データガバナンス機能に同期する
import papermill as pm
pm.execute_notebook(
    '/home/jovyan/WORKFLOW/FLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験パッケージの用意', PATH=tmp_path, TO_GIT = True)
)

In [ ]:
# 実験リポジトリをexperiments配下にクローンする
%cd ~/
!datalad clone -d . $remote_http_url ./experiments/$repository_title
!rm -rf $tmp_path

### - 3.2 pipeline.jsonに実験名を追記する

In [ ]:
import json

with open('/home/jovyan/experiments/pipeline.json', 'r') as f:
    pipeline = json.load(f)

pipeline.append(repository_title)

with open('/home/jovyan/experiments/pipeline.json', 'w') as f:
    json.dump(pipeline, f, indent = 4)

## 4. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import papermill as pm

%cd ~/

pm.execute_notebook(
    'WORKFLOW/FLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験用リポジトリの作成', IS_RECURSIVE = False, TO_GIT = True)
)

## 5. 実験環境にアクセスする

以下のうち、いずれかの手順を実施してください。

- 標準的な実験活動の場合：「5-A. 標準環境での実験」
- 高性能実験環境を利用する場合：「5-B. 高性能実験環境での実験」

### - 5-A. 標準環境での実験

以下のセルを実行した後に出力されるリンクをクリックして実験に移ってください。

In [ ]:
import urllib

print(
    "https://binder.cs.rcos.nii.ac.jp/v2/git/" + urllib.parse.quote(remote_http_url, safe='') + "/HEAD?filepath=experiment.ipynb"
)

### - 5-B. 高性能実験環境での実験

以下のセルを実行し、「高性能実験環境と正常に接続されています」と表示されることを確認してください。  
表示されない場合、[高性能実験環境を準備する](../01_preparation_phase/base_setup_data_analysis_tools.ipynb) の手順を実施してください。

In [ ]:
!ssh mdx "echo 高性能実験環境と正常に接続されています"

高性能実験環境の接続先情報を取得します。以下のセルを実行してください。

In [ ]:
mdx_host = ''
with open('/home/jovyan/.ssh/config', 'r') as f:
    tmp = f.read()
    tmp = tmp[tmp.find('mdx'):]
    tmp = tmp[tmp.find('Hostname ') + len('Hostname '):]
    mdx_host = tmp[:tmp.find('\n')]

高性能実験環境に実験パッケージ情報を転送します。以下のセルを実行してください。

In [ ]:
%cd ~/
research_project = !git config --get remote.origin.url
research_project = research_project[0].split('/')[-1].replace('.git','')

In [ ]:
%cd ~/
!ssh mdx mkdir -p $research_project
!ssh mdx mkdir -p $research_project/$repository_title
cmd = "'cd " + research_project + "/" + repository_title +"; git clone " + remote_http_url + " package'"
!ssh mdx $cmd

高性能実験環境においてJupyterLabを起動します。以下のセルを実行してください。  
実行後、表示されるリンクをクリックすると、パスワードの入力が求められますので「gpu-jupyter」と入力してLoginしてください。

In [ ]:
from IPython.display import clear_output

# mdx環境のグループ情報とホームディレクトリを取得する
groupname = !ssh mdx id -g -n
gid = !ssh mdx id -g
mdx_home = !ssh mdx 'echo $HOME'
mdx_home = mdx_home[0]

# Dockerfileのみ配置したビルドコンテキストを用意
%cd ~/
cmd = "mkdir -p build-context/" + repository_title
!ssh mdx $cmd
cmd = "cp " + research_project + "/" + repository_title + "/package/Dockerfile build-context/" + repository_title + "/Dockerfile"
!ssh mdx $cmd

# 権限を持たせるためmdx環境のグループ名とGIDを渡してJupyterLabコンテナイメージを作成する
cmd = "docker build -t dg/jupyterlab:1.1 \
--build-arg GROUPNAME=" + groupname[0] + " --build-arg GID=" + gid[0] + " ./build-context/" + repository_title
!ssh mdx $cmd

# JupyterLabコンテナの起動
# mdx上の研究プロジェクトデータと当該実験の実験パッケージ内容をマウント
mdx_path = mdx_home + "/" + research_project + "/" + repository_title + "/package/"
jupyter_path = "/home/jovyan/"
cmd = "docker run \
-v " + mdx_path + ".datalad:" + jupyter_path + ".datalad \
-v " + mdx_path + "experiment.ipynb:" + jupyter_path + "experiment.ipynb \
-v " + mdx_path + "EX-WORKFLOW:" + jupyter_path + "EX-WORKFLOW \
-v " + mdx_path + ".git:" + jupyter_path + ".git \
-v " + mdx_path + ".gitattributes:" + jupyter_path + ".gitattributes \
-v " + mdx_path + ".gitignore:" + jupyter_path + ".gitignore \
-v " + mdx_path + "input_data:" + jupyter_path + "input_data \
-v " + mdx_path + "meta_data.json:" + jupyter_path + "meta_data.json \
-v " + mdx_path + "output_data:" + jupyter_path + "output_data \
-v " + mdx_path + "source:" + jupyter_path + "source \
-v " + mdx_home + "/" + research_project +":" + jupyter_path + "research_project_data \
--gpus all -d -P dg/jupyterlab:1.1"
container_id = !ssh mdx $cmd
container_id = container_id[0][:12]

# ホストマシン上で動的に割り当てられたポートを取得
cmd = "docker ps -l"
port = !ssh mdx $cmd
port = port[1]
port = port[port.find("0.0.0.0:") + 8: port.find("->8888/tcp")]

# JupyterLab利用のためのURLを表示
url = "http://" + params['mdxDomain'] + ":" + str(port) + "/notebooks/experiment.ipynb"
print(url)

## 6. ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../base_FLOW.ipynb)からトップページに遷移できます。  